Testing out some systems to split the Infectious Pressure into home and nonhome locations and to send the infected back to their home patch 


In [2]:
import pandas as pd
import numpy as np
from pySODM.models.base import ODE, JumpProcess

In [3]:
# Initial conditions

OD = np.array([[0.75, 0.15, 0.10],  # this being a list is important for the calculation of S_fraction!
      [0.25, 0.5, 0.25], 
      [0, 0, 1]])

S = np.array([100, 50, 20]) # this being a list is important!
I_nonhome = [10, 5, 5]
f_h = 0.5
beta_t = 0.8

T = S + I

NameError: name 'I' is not defined

In [22]:
# Distributing the individuals home --> work

ODmatrixT_ndarray = np.transpose(OD)
print("OD", OD)
print("ODmatrixT_ndarray", ODmatrixT_ndarray)


S_mob = ODmatrixT_ndarray @ S
print("S", S)
print("S_mob", S_mob)

# this represents the fraction of susceptibles from each home location (i) at each destination (j). It's calculated by OD * S / S_mob
# the where(S_mob>0) is added to avoid dividing by 0

S_fraction = np.divide(OD * S[:, np.newaxis], S_mob, where=(S_mob > 0))  # Prevent division by zero
print("S_fraction", S_fraction)

# Step 3: Distribute new infections back to home locations based on S_fraction
# The new infections at non-home locations will be multiplied by the fraction of susceptibles from each home location
infections_home = S_fraction @ I_nonhome # (1 x 3)

print("\nInfections distributed back to home locations:")
print(infections_home)

OD [[0.75 0.15 0.1 ]
 [0.25 0.5  0.25]
 [0.   0.   1.  ]]
ODmatrixT_ndarray [[0.75 0.25 0.  ]
 [0.15 0.5  0.  ]
 [0.1  0.25 1.  ]]
S [100  50  20]
S_mob [87.5 40.  42.5]
S_fraction [[0.85714286 0.375      0.23529412]
 [0.14285714 0.625      0.29411765]
 [0.         0.         0.47058824]]

Infections distributed back to home locations:
[11.62289916  6.02415966  2.35294118]


In [45]:
# I need to test what happens if I calculate infections_home based on the IP_home and IP_nonhome to see whether this can be used during the rates function or whether it needs to be calculated during the apply_transitions function
I = [10, 5, 5]
T_mob = ODmatrixT_ndarray @ T
sigma = 4

# the where(S_mob>0) is added to avoid dividing by 0
S_fraction = np.divide(OD * S[:, np.newaxis], S_mob, where=(S_mob > 0))  # Prevent division by zero
print("S_fraction", S_fraction) # 3x3 showing from where to where the S people went and what fraction that represents

# Compute the infectious presssure at home and at work: 
IP_home = [f_h*beta_t * (I/T)*np.ones(T.shape), f_h*beta_t * (I/T)*np.ones(T.shape)]
print("IP_home", IP_home)
IP_nonhome = [(1-f_h)*beta_t*(I/T_mob)*np.ones(T.shape), (1-f_h)*beta_t*(I/T_mob)*np.ones(T.shape)]
print("IP_nonhome", IP_nonhome)

# Redistribute infections from non-home locations back to home locations
infections_to_home_sero1 = S_fraction @ np.array(IP_nonhome[0]).sum(axis=0)  # Total infections at non-home locations
infections_to_home_sero2 = S_fraction @ np.array(IP_nonhome[1]).sum(axis=0)  # Total infections at non-home locations

print("infections_to_home_sero1", infections_to_home_sero1)
print("infections_to_home_sero2", infections_to_home_sero2)


# what would come out of the rate without infections_to_home: 
print("rate S:", [IP_home[0] + IP_nonhome[0],  
              IP_home[1] + IP_nonhome[1]]) # the result is 2 arrays of 1x3 - one per serotype

# the alternative would be to use infections_home directly in the rate: 
print("rate S:", [IP_home[0] - infections_to_home[0],  # Adjust for home infections directly
              IP_home[1] - infections_to_home[1]],)


# just as a comparison I would like to see the shape for the rate: 
print([(1/sigma)*np.ones(T.shape),]) # (1x3)

print(infections_to_home[0],infections_to_home[1] )

S_fraction [[0.85714286 0.375      0.23529412]
 [0.14285714 0.625      0.29411765]
 [0.         0.         0.47058824]]
IP_home [array([0.03636364, 0.03636364, 0.08      ]), array([0.03636364, 0.03636364, 0.08      ])]
IP_nonhome [array([0.04155844, 0.04545455, 0.04020101]), array([0.04155844, 0.04545455, 0.04020101])]


ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [38]:
print(np.array(IP_nonhome))
print(np.array(IP_nonhome).sum(axis=0))

[[0.04155844 0.04545455 0.04020101]
 [0.04155844 0.04545455 0.04020101]]
[0.08311688 0.09090909 0.08040201]


In [39]:
IP_nonhome_stacked = np.array(IP_nonhome).reshape(-1, 2)  # Reshape if necessary to ensure correct dimensions
print("IP_nonhome_stacked", IP_nonhome_stacked)
infections_home = S_fraction @ IP_nonhome_stacked 
print("infections_home", infections_home)

IP_nonhome_stacked [[0.04155844 0.04545455]
 [0.04020101 0.04155844]
 [0.04545455 0.04020101]]
infections_home [[0.06139209 0.06400451]
 [0.04443153 0.04429136]
 [0.02139037 0.01891812]]


*Einstein notation from Tijs testing*

step 1: create the dataframes  
step 2: test the multiplication



In [5]:
# I want to check structures for the einstein notation. I need to add a psi variable to the infection parameter of the secondary infected Is I12 and I21. So I need to check whether the multiplication would still work

S = np.array([[80, 10, 1000],
              [10, 90, 4000]]) # susceptibles per age, per location

I = np.array([[10, 0, 0],
              [0, 1, 0]]) # age x location

R = np.array([[0,0,0], 
              [0,0,0]])

OD_2d = np.array([[0.6, 0.2, 0.20],
               [0.0, 1, 0.0],
               [0, 0, 1]]) # OD matrix per leeftijdsgroep 

OD_3d = np.array([[[0.6, 0.2, 0.20],
               [0.0, 1, 0.0],
               [0, 0, 1]], 
               
               [[0.6, 0.2, 0.20],
               [0.0, 1, 0.0],
               [0, 0, 1]]]) # OD matrix per leeftijdsgroep 

print("OD", OD.shape) # age x departures x arrivals (2x3x3) jki

N = np.array([[[10, 5],
               [5, 10]],

              [[10, 5], 
              [5, 10]],

              [[10, 5],
              [5, 10]]
              ]) # contact matrix

print("N", N.shape) # locations x ages x ages  (3, 2, 2) ilk


T = S + I + R
f_h = 0.8
beta = 0.8

# compute visiting population
# Perform the multiplication: summing over the departure locations (axis 1)
# 'ajk,aj->ak' means:
# - 'a' is the age group,
# - 'j' is the departure location we sum over,
# - 'k' is the destination location.
I_mob_3d = np.einsum('ajk,aj->ak', OD_3d, I)
print("I_mob_3d", I_mob_3d)
print("I_mob_3d", I_mob_3d.shape) #(2x3) ages x arrivals - identical to 2d seeing as mobility was identical

I_mob_2d = np.einsum('jk, aj -> ak', OD_2d, I)
print("I_mob_2d", I_mob_2d)
print("I_mob_2d", I_mob_2d.shape) #(2x3) ages x arrivals

print("I", I.shape) # (2x3)

OD_3d_altered = np.array([[[0.6, 0.2, 0.20],
               [0.0, 1, 0.0],
               [0, 0, 1]], 
               
               [[0.6, 0.2, 0.2],
               [0.5, 0.5, 0],
               [0, 0, 1]]]) # OD matrix per leeftijdsgroep 

I_mob_3d_altered = np.einsum('ajk,aj->ak', OD_3d_altered, I)
print("I_mob_3d", I_mob_3d_altered)
print("I_mob_3d", I_mob_3d_altered.shape)

print(S.shape[0])
# this is working!

OD (3, 3)
N (3, 2, 2)
I_mob_3d [[6. 2. 2.]
 [0. 1. 0.]]
I_mob_3d (2, 3)
I_mob_2d [[6. 2. 2.]
 [0. 1. 0.]]
I_mob_2d (2, 3)
I (2, 3)
I_mob_3d [[6.  2.  2. ]
 [0.5 0.5 0. ]]
I_mob_3d (2, 3)
2


In [13]:
# now let's check whether this works when we add the contact matrix -- THIS IS FOR MY MODEL CODE -- SEE BELOW FOR TIJS' EXAMPLE DATA
I1 = I
I2 = I
I12 = I
I21 = I
psi = 3

        # 'ajk,aj->ak' means:
        # - 'a' is the age group,
        # - 'j' is the departure location we sum over,
        # - 'k' is the destination location.
        
print("ODmatrix", OD_3d.shape) # (2, 3, 3) age x location x location

I1_mob = np.einsum('ajk,aj->ak', OD_3d, I1) # resulting is ages x destination location
I2_mob = np.einsum('ajk,aj->ak', OD_3d, I2)
I12_mob = np.einsum('ajk,aj->ak', OD_3d, I12)
I21_mob = np.einsum('ajk,aj->ak', OD_3d, I21)
T_mob = np.einsum('ajk,aj->ak', OD_3d, T)


frac = (I1+ psi*I21)/T
print("frac.shape", frac.shape) # (2 , 3) --> age x location 
print("N.shape", N.shape) # (3, 2, 2) -->  location destination x age x age

# a is age - in N it represents from whom the interaction comes
# l is location destination
# i is age - in N it represent with whom the interaction is (in this case our infectious people)
lambda_home_sero1 =  np.einsum('il,lai-> al', (I1+ psi*I21)/T, N)  ## checkt he dimensions of N
lambda_home_sero2 =  np.einsum('il,lai->al', (I2+ psi*I12)/T, N)

print("lambda_home_sero1 shape", lambda_home_sero1.shape) # (2,3) age x location

# - a is age  --> refers to ages of the susceptibles
# - j is location origin
# - k is location destination
# - i is age -->  infectious 
lambda_visit_sero1 = np.einsum ('ajk, ik, kai->ij' , OD_3d , (I1_mob + psi*I21_mob)/T_mob , N)
lambda_visit_sero2 = np.einsum ('ajk, ik, kai->ij' , OD_3d , (I2_mob + psi*I12_mob)/T_mob , N)

ODmatrix (2, 3, 3)
frac.shape (2, 3)
N.shape (3, 2, 2)
lambda_home_sero1 shape (2, 3)


In [31]:
# I will test my implementation with Tijs' example data 
beta = 0.03
f_v = 0.10

S = np.array([[80, 10, 1000],
              [10, 90, 4000]]) # susceptibles per age, per location

I = np.array([[10, 0, 0],
              [0, 0, 0]]) # age x location

R = np.array([[0,0,0], 
              [0,0,0]])

T = S+I+R

OD_2d = np.array([[0.6, 0.2, 0.20],
               [0.0, 1, 0.0],
               [0, 0, 1]]) # OD matrix per leeftijdsgroep 

OD_3d = np.array([[[0.6, 0.2, 0.20],
               [0.0, 1, 0.0],
               [0, 0, 1]], 

               [[0.6, 0.2, 0.20],
               [0.0, 1, 0.0],
               [0, 0, 1]]]) # OD matrix per leeftijdsgroep 

print("OD", OD_3d.shape) # age x departures x arrivals (2x3x3) jki

N_2d = np.array([[10, 5],
               [5, 10]])

N = np.array([[[10, 5],
               [5, 10]],
              [[10, 5], 
              [5, 10]],
              [[10, 5],
              [5, 10]]
              ]) # contact matrix

print("N", N.shape) # locations x ages x ages  (3, 2, 2) ilk
print("I", I.shape)


######################## FOR A 3D N AND M MATRIX ########################  -- not sure yet whether this is correct

I_mob = np.einsum('ajk,aj->ak', OD_3d, I) # resulting is ages x destination location
S_mob = np.einsum('ajk,aj->ak', OD_3d, S)
T_mob = np.einsum('ajk,aj->ak', OD_3d, T)

print("I_mob", I_mob.shape) # (2, 3) age x location
print("I_mob", I_mob)
print("I_mob", T_mob.shape)
print("T_mob", T_mob)
print("S_mob", S_mob)

## all good so far ## - numbers are correct
lambda_home=  beta * (1-f_v) * np.einsum('il,lai-> al', (I)/T, N)  # 2 x 3
print("lambda_home", lambda_home)
lambda_visit = beta * (f_v) * np.einsum ('ajk, ik, kai -> aj' , OD_3d , (I_mob)/T_mob , N) # 2x3
print("lambda_visit", lambda_visit)

# calculate the new infections
I_new_home = lambda_home * S
print("I_new_home", I_new_home) # --> THIS IS CORRECT -- same numbers as for the 2d example 

I_new_visit = lambda_visit * S
print("I_new_visit", I_new_visit) # --> THIS IS CORRECT -- same numbers as for the 2d example 


OD (2, 3, 3)
N (3, 2, 2)
I (2, 3)
I_mob (2, 3)
I_mob [[6. 2. 2.]
 [0. 0. 0.]]
I_mob (2, 3)
T_mob [[  54.   28. 1018.]
 [   6.   92. 4002.]]
S_mob [[  48.   26. 1016.]
 [   6.   92. 4002.]]
lambda_home [[0.03  0.    0.   ]
 [0.015 0.    0.   ]]
lambda_visit [[2.44035925e-03 2.14285714e-03 5.89390963e-05]
 [1.22017962e-03 1.07142857e-03 2.94695481e-05]]
I_new_home [[2.4  0.   0.  ]
 [0.15 0.   0.  ]]
I_new_visit [[0.19522874 0.02142857 0.0589391 ]
 [0.0122018  0.09642857 0.11787819]]


In [30]:
######################## FOR A 2D N AND M MATRIX ########################
I_mob = np.einsum('jk,aj->ak', OD_2d, I) # origin x destination * age x destination
S_mob = np.einsum('jk,aj->ak', OD_2d, S)
T_mob = np.einsum('jk,aj->ak', OD_2d, T)

print("I_mob", I_mob.shape) # (2, 3) age x location
print("I_mob", I_mob)
print("I_mob", T_mob.shape)
print("T_mob", T_mob)
print("S_mob", S_mob)

## all good so far ## - numbers are correct
lambda_home=  beta * (1-f_v) * np.einsum('il, ai-> al', (I)/T, N_2d)  # 2 x 3
print("lambda_home", lambda_home)
lambda_visit = beta * (f_v) * np.einsum ('jk, ik, ai -> aj' , OD_2d , (I_mob)/T_mob , N_2d) # 2x3
print("lambda_visit", lambda_visit)

# calculate the new infections
I_new_home = lambda_home * S
print("I_new_home", I_new_home) # --> THIS IS CORRECT

I_new_visit = lambda_visit * S
print("I_new_visit", I_new_visit) # --> THIS IS CORRECT


I_mob (2, 3)
I_mob [[6. 2. 2.]
 [0. 0. 0.]]
I_mob (2, 3)
T_mob [[  54.   28. 1018.]
 [   6.   92. 4002.]]
S_mob [[  48.   26. 1016.]
 [   6.   92. 4002.]]
lambda_home [[0.03  0.    0.   ]
 [0.015 0.    0.   ]]
lambda_visit [[2.44035925e-03 2.14285714e-03 5.89390963e-05]
 [1.22017962e-03 1.07142857e-03 2.94695481e-05]]
I_new_home [[2.4  0.   0.  ]
 [0.15 0.   0.  ]]
I_new_visit [[0.19522874 0.02142857 0.0589391 ]
 [0.0122018  0.09642857 0.11787819]]


In [46]:
# FINAL CHECK TO SEE IF I MAKE M IN LOC X LOC X AGE IF MY CODE ADAPTATION WORKS --> also works!

# I will test my implementation with Tijs' example data 
beta = 0.03
f_v = 0.10

S = np.array([[80, 10, 1000],
              [10, 90, 4000]]) # susceptibles per age, per location

I = np.array([[10, 0, 0],
              [0, 0, 0]]) # age x location

R = np.array([[0,0,0], 
              [0,0,0]])

T = S+I+R


OD_child = np.array([[0.6, 0.2, 0.20],
               [0.0, 1, 0.0],
               [0, 0, 1]])

OD_adult = np.array([[0.6, 0.2, 0.20],
               [0.0, 1, 0.0],
               [0, 0, 1]])

print("OD_child", OD_child)
OD_3d_reshaped = np.stack((OD_child, OD_adult), axis = 2)

print("OD_3d_reshaped", OD_3d_reshaped.shape) # departures x arrivals x age (3x3x2) 


N_A = np.array([[10, 5],
                [5, 10]])
N_B = np.array([[10, 5],
                [5, 10]])
N_C = np.array([[10, 5],
                [5, 10]])

N_reshaped = np.stack((N_A, N_B, N_C), axis=2)

print("N", N.shape) # locations x ages x ages  (3, 2, 2) 
print("N_reshaped", N_reshaped.shape) # age x age x location (2x2x3) 

print("I", I.shape)


I_mob = np.einsum('jka,aj->ak', OD_3d_reshaped, I) # resulting is ages x destination location
S_mob = np.einsum('jka,aj->ak', OD_3d_reshaped, S)
T_mob = np.einsum('jka,aj->ak', OD_3d_reshaped, T)

print("I_mob", I_mob.shape) # (2, 3) age x location
print("I_mob", I_mob)
print("I_mob", T_mob.shape)
print("T_mob", T_mob)
print("S_mob", S_mob)

## all good so far ## - numbers are correct
lambda_home=  beta * (1-f_v) * np.einsum('il,ail-> al', (I)/T, N_reshaped)  # 2 x 3
print("lambda_home", lambda_home)
lambda_visit = beta * (f_v) * np.einsum ('jla, il, ail -> aj' , OD_3d_reshaped , (I_mob)/T_mob , N_reshaped) # 2x3
print("lambda_visit", lambda_visit)

# calculate the new infections
I_new_home = lambda_home * S
print("I_new_home", I_new_home) # --> THIS IS CORRECT -- same numbers as for the 2d example 

I_new_visit = lambda_visit * S
print("I_new_visit", I_new_visit) # --> THIS IS CORRECT -- same numbers as for the 2d example 

OD_child [[0.6 0.2 0.2]
 [0.  1.  0. ]
 [0.  0.  1. ]]
OD_3d_reshaped (3, 3, 2)
N (3, 2, 2)
N_reshaped (2, 2, 3)
I (2, 3)
I_mob (2, 3)
I_mob [[6. 2. 2.]
 [0. 0. 0.]]
I_mob (2, 3)
T_mob [[  54.   28. 1018.]
 [   6.   92. 4002.]]
S_mob [[  48.   26. 1016.]
 [   6.   92. 4002.]]
lambda_home [[0.03  0.    0.   ]
 [0.015 0.    0.   ]]
lambda_visit [[2.44035925e-03 2.14285714e-03 5.89390963e-05]
 [1.22017962e-03 1.07142857e-03 2.94695481e-05]]
I_new_home [[2.4  0.   0.  ]
 [0.15 0.   0.  ]]
I_new_visit [[0.19522874 0.02142857 0.0589391 ]
 [0.0122018  0.09642857 0.11787819]]


In [48]:
### NOW I NEED TO ADD A BETA THAT IS DIFFERENT PER AGE GROUP  ####

beta_0 = np.array([0.2, 0.10])
beta_1 = 0.5

f_v = 0.10

S = np.array([[80, 10, 1000],
              [10, 90, 4000]]) # susceptibles per age, per location

I = np.array([[10, 0, 0],
              [0, 0, 0]]) # age x location

R = np.array([[0,0,0], 
              [0,0,0]])

T = S+I+R


OD_child = np.array([[0.6, 0.2, 0.20],
               [0.0, 1, 0.0],
               [0, 0, 1]])

OD_adult = np.array([[0.6, 0.2, 0.20],
               [0.0, 1, 0.0],
               [0, 0, 1]])

print("OD_child", OD_child)
OD_3d_reshaped = np.stack((OD_child, OD_adult), axis = 2)

print("OD_3d_reshaped", OD_3d_reshaped.shape) # departures x arrivals x age (3x3x2) 


N_A = np.array([[10, 5],
                [5, 10]])
N_B = np.array([[10, 5],
                [5, 10]])
N_C = np.array([[10, 5],
                [5, 10]])

N_reshaped = np.stack((N_A, N_B, N_C), axis=2)

print("N", N.shape) # locations x ages x ages  (3, 2, 2) 
print("N_reshaped", N_reshaped.shape) # age x age x location (2x2x3) 

I_mob = np.einsum('jla,aj->al', OD_3d_reshaped, I) # resulting is ages x destination location
S_mob = np.einsum('jla,aj->al', OD_3d_reshaped, S)
T_mob = np.einsum('jla,aj->al', OD_3d_reshaped, T)

## all good so far ## - numbers are correct
lambda_home=  beta_0 * (1-f_v) * np.einsum('il,ail-> al', (I)/T, N_reshaped)  # 2 x 3
print("lambda_home", lambda_home)
lambda_visit = beta_0 * (f_v) * np.einsum ('jla, il, ail -> aj' , OD_3d_reshaped , (I_mob)/T_mob , N_reshaped) # 2x3
print("lambda_visit", lambda_visit)

# calculate the new infections
I_new_home = lambda_home * S
print("I_new_home", I_new_home) # --> THIS IS CORRECT -- same numbers as for the 2d example 

I_new_visit = lambda_visit * S
print("I_new_visit", I_new_visit) # --> THIS IS CORRECT -- same numbers as for the 2d example 

OD_child [[0.6 0.2 0.2]
 [0.  1.  0. ]
 [0.  0.  1. ]]
OD_3d_reshaped (3, 3, 2)
N (3, 2, 2)
N_reshaped (2, 2, 3)
lambda_home [[1.  0.  0. ]
 [0.5 0.  0. ]]


ValueError: operands could not be broadcast together with shapes (2,) (2,3) 

In [52]:
beta = np.array([0,1])
npeinstoem = np.array([[10,5,1], [10,5,1]])

print("beta", beta.shape) #(2,)
print("npeinstoem", npeinstoem.shape) #(2,3)

result = beta[:, np.newaxis] * npeinstoem
print(beta[:, np.newaxis])
print(beta[:, np.newaxis].shape) # (2,1)

print("result", result)

beta (2,)
npeinstoem (2, 3)
[[0]
 [1]]
(2, 1)
result [[ 0  0  0]
 [10  5  1]]


## Stochastic mobility -> Multinomial distribution
I would like to check how I can make my mobility stochastic and whether I can still use the Einstein notation or not.

In [4]:
S = np.array([[80, 10, 1000],
              [10, 90, 4000]]) # susceptibles per age, per location

I = np.array([[10, 0, 0],
              [0, 0, 0]]) # age x location

R = np.array([[0,0,0], 
              [0,0,0]])

T = S+I+R


OD_child = np.array([[0.6, 0.2, 0.20],
               [0.0, 1, 0.0],
               [0, 0, 1]])

OD_adult = np.array([[0.6, 0.2, 0.20],
               [0.0, 1, 0.0],
               [0, 0, 1]])

print("OD_child", OD_child)
OD_3d_reshaped = np.stack((OD_child, OD_adult), axis = 2)
print(OD_3d_reshaped.shape) # (3, 3, 2)

OD_child [[0.6 0.2 0.2]
 [0.  1.  0. ]
 [0.  0.  1. ]]
(3, 3, 2)


In [36]:
# with a vectorized approach: 
# I is the infectious compartment with shape (age, location) => (2, 3)
# ODmatrix is the origin-destination matrix with shape (origin, destination, age) => (3, 3, 2)

I = np.array([[10, 0, 0],
              [1, 0, 0]]) # age x location

OD_child = np.array([[0.6, 0.2, 0.20],
               [0.0, 1, 0.0],
               [0, 0, 1]])

OD_adult = np.array([[0.6, 0.2, 0.20],
               [0.0, 1, 0.0],
               [0, 0, 1]])

print("OD_child", OD_child)
OD_3d_reshaped = np.stack((OD_child, OD_adult), axis = 2)

# Reshape I and ODmatrix for vectorized multinomial sampling
I_flat = I.flatten()  # Flatten I into a 1D array of shape (agexlocations = 6,) by transposing (to match location-first order)
print("I_flat", I_flat)
ODmatrix_flat = OD_3d_reshaped.reshape(3, 3, 2).transpose(2, 0, 1).reshape(6, 3)  # Reshape ODmatrix to (6, 3) for each (origin, destination) pair for each age group
print("ODmatrix_flat", ODmatrix_flat)

# Perform multinomial draws for each origin-destination-age group combination
transitions = np.array([rng.multinomial(I_flat[i], ODmatrix_flat[i]) for i in range(I_flat.size)])
print("I_flat[1]", I_flat[0]) # 10
print("ODmatrix_flat[1]", ODmatrix_flat[0]) #[0.6 0.2 0.2]

print("I_flat[3]", I_flat[3]) # 1
print("ODmatrix_flat[3]", ODmatrix_flat[3]) #[0.6 0.2 0.2]

# Reshape the result back to the original age x location format
I_mobility = transitions.reshape(2, 3, 3).sum(axis=1) # sum over departure locations
print("I_mobility", I_mobility)

OD_child [[0.6 0.2 0.2]
 [0.  1.  0. ]
 [0.  0.  1. ]]
I_flat [10  0  0  1  0  0]
ODmatrix_flat [[0.6 0.2 0.2]
 [0.  1.  0. ]
 [0.  0.  1. ]
 [0.6 0.2 0.2]
 [0.  1.  0. ]
 [0.  0.  1. ]]
I_flat[1] 10
ODmatrix_flat[1] [0.6 0.2 0.2]
I_flat[3] 1
ODmatrix_flat[3] [0.6 0.2 0.2]
I_mobility [[4 0 6]
 [1 0 0]]


In [48]:
OD_child = np.array([[0.6, 0.2, 0.20],
               [0.0, 1, 0.0],
               [0, 0, 1]])

OD_adult = np.array([[0.6, 0.2, 0.20],
               [0.0, 1, 0.0],
               [0, 0, 1]])

OD_3d_reshaped = np.stack((OD_child, OD_adult), axis = 2)

age_dim, loc_dim = I.shape  # Shape of I: (age, location)
origin_dim, destination_dim, age_dim_od = OD_3d_reshaped.shape  # Shape of ODmatrix: (origin, destination, age)

# Ensure that the age dimension matches between I and ODmatrix
assert age_dim == age_dim_od, "Age dimensions of I and ODmatrix do not match."

# Flatten the infectious compartment (I) and reshape the ODmatrix accordingly
I_flat = I.flatten()  # Flatten I to (age * location,)
print("I_flat", I_flat)
ODmatrix_flat = OD_3d_reshaped.transpose(2, 0, 1).reshape(age_dim * loc_dim, destination_dim)
print("ODmatrix_flat", ODmatrix_flat)

# Perform multinomial draws for each origin-age combination in one step
transitions = np.array([rng.multinomial(I_flat[i], ODmatrix_flat[i]) for i in range(I_flat.size)])

# Sum over the destinations (axis=1) to aggregate infections at each location and reshape to (age, location)
I_mobility = transitions.reshape(age_dim,loc_dim, loc_dim).sum(axis=1)
print(I_mobility)

I_flat [10  0  0  1  0  0]
ODmatrix_flat [[0.6 0.2 0.2]
 [0.  1.  0. ]
 [0.  0.  1. ]
 [0.6 0.2 0.2]
 [0.  1.  0. ]
 [0.  0.  1. ]]
[[6 1 3]
 [1 0 0]]


In [57]:
# with a function: 
def stochastic_mobility(state, OD, rng):
    age_dim, loc_dim = I.shape  # Shape of I: (age, location)
    origin_dim, destination_dim, age_dim_od = OD.shape  # Shape of ODmatrix: (origin, destination, age)

    # Ensure that the age dimension matches between I and ODmatrix
    assert age_dim == age_dim_od, "Age dimensions of I and ODmatrix do not match."

    # Flatten the infectious compartment (I)
    state_flat = state.flatten()  # Shape: (age * location,)
    
    # Reshape ODmatrix for multinomial sampling
    ODmatrix_flat = OD.transpose(2, 0, 1).reshape(age_dim * loc_dim, destination_dim)

    # Perform multinomial draws
    transitions = np.array([rng.multinomial(state_flat[i], ODmatrix_flat[i]) for i in range(state_flat.size)])

    # Aggregate transitions and reshape to (age, location)
    state_mobility = transitions.reshape(age_dim, loc_dim, loc_dim).sum(axis=1)

    return state_mobility  # Return the mobility result

rng = np.random.default_rng()

I_mob = stochastic_mobility(I, OD_3d_reshaped, rng)
print(I_mob)

[[7 2 1]
 [0 0 1]]


# Run repeated simulations to check if results still align with Tijs' example
I will use the initial conditions and parameter values given in the example and run a repeated simulation of the stochastic mobility process. 
The output should be identical on average. 

In [65]:
#########################
# set up the model
#########################

## INITIAL CONDITIONS ##
beta_0 = np.array([0.03, 0.03])
f_v = 0.10

S = np.array([[80, 10, 1000],
              [10, 90, 4000]]) # susceptibles per age, per location

I = np.array([[10, 0, 0],
              [0, 0, 0]]) # age x location

R = np.array([[0,0,0], 
              [0,0,0]])

T = S+I+R

## OD MATRIX ##
OD_child = np.array([[0.6, 0.2, 0.20],
               [0.0, 1, 0.0],
               [0, 0, 1]])

OD_adult = np.array([[0.6, 0.2, 0.20],
               [0.0, 1, 0.0],
               [0, 0, 1]])

OD_3d_reshaped = np.stack((OD_child, OD_adult), axis = 2)

## CONTACT MATRIX ##
N_A = np.array([[10, 5],
                [5, 10]])
N_B = np.array([[10, 5],
                [5, 10]])
N_C = np.array([[10, 5],
                [5, 10]])

N_reshaped = np.stack((N_A, N_B, N_C), axis=2)

################################
# STOCHASTIC MOBILITY FUNCTION
################################
def stochastic_mobility(state, OD, rng):
    age_dim, loc_dim = I.shape  # Shape of I: (age, location)
    origin_dim, destination_dim, age_dim_od = OD.shape  # Shape of ODmatrix: (origin, destination, age)

    # Ensure that the age dimension matches between I and ODmatrix
    assert age_dim == age_dim_od, "Age dimensions of I and ODmatrix do not match."

    # Flatten the infectious compartment (I)
    state_flat = state.flatten()  # Shape: (age * location,)
    
    # Reshape ODmatrix for multinomial sampling
    ODmatrix_flat = OD.transpose(2, 0, 1).reshape(age_dim * loc_dim, destination_dim)

    # Perform multinomial draws
    transitions = np.array([rng.multinomial(state_flat[i], ODmatrix_flat[i]) for i in range(state_flat.size)])

    # Aggregate transitions and reshape to (age, location)
    state_mobility = transitions.reshape(age_dim, loc_dim, loc_dim).sum(axis=1)

    return state_mobility  # Return the mobility result

##############################################
# RUN INFECTION PROCESS AT HOME AND NON-HOME - REPEATED SIMULATION
##############################################

# Number of repetitions
n_repetitions = 100

# Arrays to store results for averaging
I_new_home_results = np.zeros((2, 3))  # Adjust shape based on your S shape
I_new_visit_results = np.zeros((2, 3))  # Adjust shape based on your S shape

# Main loop to run the process 100 times
for _ in range(n_repetitions):
    # Run stochastic mobility for I and T
    I_mob = stochastic_mobility(I, OD_3d_reshaped, rng)
    T_mob = stochastic_mobility(T, OD_3d_reshaped, rng)

    # Calculate lambda_home and lambda_visit
    lambda_home = beta_0[:, np.newaxis] * (1 - f_v) * np.einsum('il, ail->al', (I) / T, N_reshaped)  # 2 x 3
    lambda_visit = beta_0[:, np.newaxis] * (f_v) * np.einsum('jla, il, ail -> aj', OD_3d_reshaped, (I_mob) / T_mob, N_reshaped)  # 2 x 3

    # Calculate new infections
    I_new_home = lambda_home * S
    I_new_visit = lambda_visit * S

    # Accumulate results
    I_new_home_results += I_new_home
    I_new_visit_results += I_new_visit

# Calculate the average results
I_new_home_avg = I_new_home_results / n_repetitions
I_new_visit_avg = I_new_visit_results / n_repetitions

# Print average results
print("Average I_new_home:\n", I_new_home_avg)
print("Average I_new_visit:\n", I_new_visit_avg)

## THIS IS CORRECT :)

Average I_new_home:
 [[2.4  0.   0.  ]
 [0.15 0.   0.  ]]
Average I_new_visit:
 [[0.19089623 0.01987392 0.06749444]
 [0.01193101 0.08943266 0.13498888]]
